In [1]:
#pip install --upgrade openai langchain langchain-openai langchain-community langgraph

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from typing import List, Dict, Any
import random
load_dotenv()

True

In [3]:
OPENAI_MODEL = "gpt-4.1-mini"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool, tool
from typing import List
import json
import os
import select
from datetime import datetime
from textwrap import dedent
from typing import Dict, List, Optional, TypeVar, Union
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic import ValidationError
from typing import Annotated, Any, Dict, List, Literal, Optional, TypedDict, TypeVar, Union
from IPython.display import Image, display
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.runnables import Runnable, RunnableConfig, RunnableLambda
from langchain_core.utils.function_calling import convert_to_openai_tool
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
import sys
import uuid
from pprint import pprint

In [5]:
prompt_template = dedent("""
### INSTRUÇÕES PARA RESPONDER À PERGUNTA DO HUMANO:

1- Entenda a DINÂMICA:

    Você é um vendedor online que responde às perguntas do cliente usando o histórico da conversa e as ferramentas disponíveis. 

2. Passos PARA RESPONDER:

    Pergunta: Pergunta do cliente.
    Reflexão: 
        a. Pense passo a passo
        b. Analise a pergunta do cliente.
        c. Revise o histórico para contexto.
        d. Verifique as ferramentas disponíveis.
    Ação: A ação a ser tomada. 
    Entrada: Dados necessários para a ação 
    Observação: Resultado da ação 
    Reflexão: Repita se necessário
    Resposta Final: Resposta para a pergunta do cliente

### Contexto:
- Idioma da Resposta: {idioma}
- E-mail do Usuário: {email_usuario}
- Data de Hoje: {data_hoje}
- Dia da Semana: {dia_da_semana}

### Histórico da conversa:
{chat_history} 

### Solicitação do cliente:
{input}

Scratchpad:
{agent_scratchpad}

===
{messages}
===

**Begin!**

Resposta:
""")


In [6]:
prompt_str = dedent("""Você é um assistente atencioso, educado e prestativo.
Sua missão é responder às perguntas do cliente com gentileza, cortesia e precisão.

## Regras de Resposta

1. Todas as informações devem ser baseadas em dados concretos. Nunca invente produtos ou preços.
2. Baseie suas respostas exclusivamente no HISTÓRICO DA CONVERSA e CONTEXTO fornecido ou nas ferramentas disponíveis.
3. Não procure ou especule informações fora do histórico ou contexto. Se não houver dados suficientes, responda:
"Desculpe, não tenho informações suficientes para responder a essa pergunta com precisão."
4. Mantenha um tom cordial, educado e informativo. Forneça respostas claras e diretas.
5. Use as ferramentas disponíveis para elaborar a resposta.
6. Se para buscar a resposta for necessário uma ferramenta que não esteja disponível, informe em sua resposta o nome dessa ferramenta.

## Instruções para Responder

Analise o histórico da conversa e responda a solicitação do cliente.

# Contexto:
- Idioma da Resposta: português
- E-mail do Usuário: mrctito@gmail.com
- Data de Hoje: 2024-12-17
- Dia da Semana: terça


# Solicitação do cliente:
Criar uma mensagem para redes sociais personalizado para 3 clientes, oferecendo um produto similar 
ao que ele comprou recentemente, mas que tenha a maior margem de lucro para a empresa.
                    
Para cada carta gerada, explique detalhadamente o porquê da escolha do produto.

---

# Scratchpad:


---

# Mensagens:  
{{messages}}

---

Resposta:""")

In [7]:
query = dedent(""" Criar uma mensagem para redes sociais personalizado para 3 clientes, 
oferecendo um produto similar ao que ele comprou recentemente, e de acordo com suas preferências pessoais, mas que tenha a maior margem 
de lucro para a empresa. 
""")
idioma = "português"
codigo_cliente = 1
email_cliente = "mrctito@gmail.com"
nome_cliente = "Paulo"
email_usuario = "mrctito@gmail.com"
nome_usuario = "Paulo"
memory_messages = None

contexto = {
    "messages": [HumanMessage(content=query)],
    "input": query,
    "idioma": idioma,
    "dia_da_semana": datetime.now().strftime("%A"),
    "data_hoje": datetime.now().strftime("%Y-%m-%d"),
    "email_cliente": email_usuario,
    "nome_cliente": nome_usuario,
    "email_usuario": email_usuario,
    "nome_usuario": nome_usuario,
    "chat_history": memory_messages if memory_messages else " ",
    "agent_scratchpad": "",
}

In [8]:
@tool
async def tool_informa_o_clima() -> str:
    """
    Útil para informar o clima atual.
    """
    return ["O clima está estável com sol."]

In [9]:
@tool
def buscar_ultimas_compras(quantidade_clientes):
    """
    Busca as últimas compras feitas por um número determinado de clientes.
    
    Args:
        quantidade_clientes: número inteiro de clientes para simulação.
    
    Retorna:
        Lista de dicionários, cada um contendo:
            - cliente_id: identificador do cliente.
            - produto_id: identificador do produto comprado.
            - data_compra: data fictícia da compra (string).
    """
    clientes_ids = [f"C{1000+i}" for i in range(quantidade_clientes)]
    produtos_ids = ["P001", "P002", "P003", "P004", "P005", "P006", "P007", "P008", "P009", "P010"]
    
    compras = []
    for cliente_id in clientes_ids:
        produto_id = random.choice(produtos_ids)
        compra = {
            "cliente_id": cliente_id,
            "produto_id": produto_id,
            "data_compra": "2025-06-25"
        }
        compras.append(compra)
    return compras

@tool
def buscar_produtos_similares(produto_id):
    """
    Busca produtos similares a um produto informado.
    
    Args:
        produto_id: string identificando o produto de referência.
    
    Retorna:
        Lista de dicionários, cada um contendo:
            - produto_id: identificador do produto similar.
            - nome: nome fictício do produto similar.
            - categoria: categoria do produto similar.
    """
    similares_map = {
        "P001": [
            {"produto_id": "P101", "nome": "Pizza Margherita Grande", "categoria": "Pizza"},
            {"produto_id": "P102", "nome": "Pizza Calabresa Grande", "categoria": "Pizza"},
            {"produto_id": "P103", "nome": "Pizza Portuguesa Grande", "categoria": "Pizza"},
        ],
        "P002": [
            {"produto_id": "P104", "nome": "Hambúrguer Tradicional", "categoria": "Hambúrguer"},
            {"produto_id": "P105", "nome": "Hambúrguer com Queijo Cheddar", "categoria": "Hambúrguer"},
            {"produto_id": "P106", "nome": "Hambúrguer Bacon Deluxe", "categoria": "Hambúrguer"},
        ],
        "P003": [
            {"produto_id": "P107", "nome": "Porção de Batata Frita Média", "categoria": "Acompanhamento"},
            {"produto_id": "P108", "nome": "Porção de Onion Rings", "categoria": "Acompanhamento"},
            {"produto_id": "P109", "nome": "Porção de Nuggets", "categoria": "Acompanhamento"},
        ],
    }
    return similares_map.get(produto_id, [
        {"produto_id": "P110", "nome": "Refrigerante Cola 600ml", "categoria": "Bebida"},
        {"produto_id": "P111", "nome": "Suco Natural Laranja 300ml", "categoria": "Bebida"},
    ])

@tool
def buscar_margens_lucro(produtos_ids):
    """
    Busca a margem de lucro simulada para uma lista de produtos.
    
    Args:
        produtos_ids: lista de strings com os identificadores dos produtos.
    
    Retorna:
        Dicionário onde a chave é o produto_id e o valor é a margem de lucro em percentual (float).
    """
    margens = {}
    for pid in produtos_ids:
        margens[pid] = round(random.uniform(15, 60), 2)
    return margens

@tool
def buscar_dados_cliente(clientes_ids):
    """
    Busca dados fictícios (nome e e-mail) para uma lista de clientes.
    
    Args:
        clientes_ids: lista de strings com os identificadores dos clientes.
    
    Retorna:
        Lista de dicionários, cada um contendo:
            - cliente_id: identificador do cliente.
            - nome: nome fictício do cliente.
            - email: e-mail fictício do cliente.
    """
    nomes = [
        "Ana Silva", "Bruno Souza", "Carla Dias", "Daniel Oliveira", "Eduardo Lima",
        "Fernanda Costa", "Gabriel Alves", "Helena Rocha", "Igor Santos", "Joana Melo"
    ]
    emails = [
        "ana@email.com", "bruno@email.com", "carla@email.com", "daniel@email.com",
        "eduardo@email.com", "fernanda@email.com", "gabriel@email.com", "helena@email.com",
        "igor@email.com", "joana@email.com"
    ]
    dados_clientes = []
    for i, cid in enumerate(clientes_ids):
        dados = {
            "cliente_id": cid,
            "nome": nomes[i % len(nomes)],
            "email": emails[i % len(emails)]
        }
        dados_clientes.append(dados)
    return dados_clientes


@tool
def buscar_preferencias_cliente_pelo_id(id_cliente):
    """
    Busca as preferências pessoais de um cliente específico pelo ID
    Input deve ser o ID do cliente
    """

    preferencias = []
    return preferencias


@tool
def buscar_preferencias_cliente_pelo_nome(nome_cliente):
    """
    Busca as preferências pessoais de um cliente específico pelo nome
    Input deve ser o nome do cliente
    """

    preferencias = []
    if nome_cliente == "Ana Silva":
        preferencias = ["comidas pesadas", "dias ensolarados"]
    elif nome_cliente == "Bruno Souza":
        preferencias = ["comidas leves"]
    elif nome_cliente == "Carla Dias":
        preferencias = ["dias chuvosos"]
    elif nome_cliente == "Daniel Oliveira": 
        preferencias = ["Pizza", "Acompanhamentos", "Bebidas"]
    elif nome_cliente == "Eduardo Lima":
        preferencias = ["Hambúrgueres", "Bebidas", "Sobremesas"]
    elif nome_cliente == "Fernanda Costa":
        preferencias = ["Pizza", "Acompanhamentos", "Bebidas"]

    return preferencias



@tool
def escolher_produto_maior_margem(produtos_similares, margens_lucro):
    """
    Seleciona o produto com a maior margem de lucro dentre uma lista de produtos similares.
    
    Args:
        produtos_similares: lista de dicionários de produtos similares.
        margens_lucro: dicionário com margens de lucro por produto_id.
    
    Retorna:
        Dicionário do produto com maior margem, contendo:
            - produto_id
            - nome
            - categoria
            - margem_lucro (float)
    """
    melhor_produto = None
    maior_margem = -1.0
    for produto in produtos_similares:
        pid = produto["produto_id"]
        margem = margens_lucro.get(pid, 0)
        if margem > maior_margem:
            maior_margem = margem
            melhor_produto = {
                **produto,
                "margem_lucro": margem
            }
    return melhor_produto


In [10]:
tools = []
tools.append(tool_informa_o_clima)
tools.append(buscar_ultimas_compras)
tools.append(buscar_produtos_similares)
tools.append(buscar_margens_lucro)
tools.append(buscar_dados_cliente)
tools.append(escolher_produto_maior_margem)
tools.append(buscar_preferencias_cliente_pelo_nome)
tools.append(buscar_preferencias_cliente_pelo_id)
tools_names = "\n\n".join([f"Ferramenta: {tool.name}\nDescrição: {tool.description}" for tool in tools])
tool_node = ToolNode(tools)

In [11]:
llm = ChatOpenAI(model=OPENAI_MODEL,
                 api_key=OPENAI_API_KEY,
                 temperature=0.7,
                 verbose=True)

In [12]:
llm_with_tools = llm.bind_tools(tools)

In [13]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_str),
        ("human", "{input}"),
        ("placeholder", "{messages}"),
    ]
)

In [14]:
assistant_runnable = primary_assistant_prompt | llm_with_tools

In [15]:
# Define State
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [16]:
# Define the function that determines whether to continue or not
def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    else:
        return "tools"


# Define the function that calls the model
async def call_model(state: State, config: RunnableConfig):
    messages = state["messages"]
    # Note: Passing the config through explicitly is required for python < 3.11
    # Since context var support wasn't added before then: https://docs.python.org/3/library/asyncio-task.html#creating-tasks

    configuration = config.get("configurable", {})
    contexto = configuration.get("contexto", {})
    user_id = configuration.get("user_id", None)
    state = {**state, "user_info": user_id, **contexto}
    
    response = await assistant_runnable.ainvoke(messages, config)
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next we pass in the path map - all the nodes this edge could go to
    ["tools", END],
)

workflow.add_edge("tools", "agent")

In [17]:
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)
thread_id = str(uuid.uuid4())
config = {
    "configurable": {
        # fetch the user's id
        "user_id": email_usuario,
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
        "contexto": contexto
    }
}

In [18]:
resultado = await graph.ainvoke(contexto, config, debug=True)

[values] {'messages': [HumanMessage(content=' Criar uma mensagem para redes sociais personalizado para 3 clientes, \noferecendo um produto similar ao que ele comprou recentemente, e de acordo com suas preferências pessoais, mas que tenha a maior margem \nde lucro para a empresa. \n', additional_kwargs={}, response_metadata={}, id='d34e48cd-2e23-4303-a953-27f242044886')]}
[updates] {'agent': {'messages': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8BZJR7UTNHaHZg15F3bcSeFa', 'function': {'arguments': '{"quantidade_clientes":3}', 'name': 'buscar_ultimas_compras'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 998, 'total_tokens': 1019, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerpr

In [19]:
pprint(resultado)

{'messages': [HumanMessage(content=' Criar uma mensagem para redes sociais personalizado para 3 clientes, \noferecendo um produto similar ao que ele comprou recentemente, e de acordo com suas preferências pessoais, mas que tenha a maior margem \nde lucro para a empresa. \n', additional_kwargs={}, response_metadata={}, id='d34e48cd-2e23-4303-a953-27f242044886'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8BZJR7UTNHaHZg15F3bcSeFa', 'function': {'arguments': '{"quantidade_clientes":3}', 'name': 'buscar_ultimas_compras'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 998, 'total_tokens': 1019, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-B

In [20]:
ultima_mensagem = resultado['messages'][-1]
print(ultima_mensagem)

content='Aqui estão as mensagens personalizadas para os 3 clientes, oferecendo um produto similar ao que compraram recentemente, baseado na maior margem de lucro disponível, embora não tenha sido possível obter preferências pessoais específicas dos clientes:\n\n---\n\nCliente: Ana Silva  \nProduto similar escolhido: Suco Natural Laranja 300ml (margem de lucro 49,52%)  \nMensagem para redes sociais:  \n"Olá Ana Silva! Sabemos que você aprecia bebidas refrescantes. Que tal experimentar nosso Suco Natural Laranja 300ml? Além de delicioso, é uma opção saudável com um sabor natural que vai conquistar seu paladar. Aproveite essa novidade com a gente!"\n\nJustificativa: O produto escolhido é o que apresenta a maior margem de lucro entre os similares (Suco Natural Laranja 300ml com 49,52%), oferecendo uma alternativa atraente e lucrativa para a empresa.\n\n---\n\nCliente: Bruno Souza  \nProduto similar escolhido: Suco Natural Laranja 300ml (margem de lucro 49,52%)  \nMensagem para redes sociai

In [21]:
print(ultima_mensagem.content)

Aqui estão as mensagens personalizadas para os 3 clientes, oferecendo um produto similar ao que compraram recentemente, baseado na maior margem de lucro disponível, embora não tenha sido possível obter preferências pessoais específicas dos clientes:

---

Cliente: Ana Silva  
Produto similar escolhido: Suco Natural Laranja 300ml (margem de lucro 49,52%)  
Mensagem para redes sociais:  
"Olá Ana Silva! Sabemos que você aprecia bebidas refrescantes. Que tal experimentar nosso Suco Natural Laranja 300ml? Além de delicioso, é uma opção saudável com um sabor natural que vai conquistar seu paladar. Aproveite essa novidade com a gente!"

Justificativa: O produto escolhido é o que apresenta a maior margem de lucro entre os similares (Suco Natural Laranja 300ml com 49,52%), oferecendo uma alternativa atraente e lucrativa para a empresa.

---

Cliente: Bruno Souza  
Produto similar escolhido: Suco Natural Laranja 300ml (margem de lucro 49,52%)  
Mensagem para redes sociais:  
"Olá Bruno Souza! Q